# Object Detection

**IoU** (Intersection over Union) is calculated as the area of overlap between two bounding boxes divided by the area of their union. 

$$ IoU = \frac{b_1 \cap b_2}{b_1 \cup b_2}, 0 \le IoU \le 1$$

While this is a good metric to evaluate object detection algorithms, it cannot be used as a loss function, because it is not differentiable when the boxes do not overlap.

To overcome this, the **GIoU** (Generalized-IoU) has been introduced

$$ GIoU = IoU - \frac{C - b_1 \cup b_2}{C} $$

It uses additionally the area of the smallest enclosing box ($C$) that contains both boxes, thus considering the relative positions of the boxes and solving the problem that non-overlapping bounding boxes have an $IoU=0$
If one box is completely inside the other, GIoU will degrade to IoU and experiments have shown that convergence can become difficult.

Next, **DIoU** (Distance-IOU)

$$ DIoU = IoU - \frac{\rho^2}{c^2} $$

**DIoU** considers the distance of the two bounding boxes ($\rho$) with respect to the enclosing box diagonal length ($c$) helping in convergence.

Finally, this has been extended to include the aspect ratio of the different bounding boxes, resulting in **CIoU** (Complete-IoU).

$$ CIoU = IoU - \frac{\rho^2}{c^2}-\alpha v $$

$\alpha v$ encapsulate the different aspect ratio of the bounding boxes.

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import jax.numpy as jnp
from jax import grad

def plot(box, color, ax):
    ax.set_xlim(0, 10)
    ax.set_ylim(0, 10)
    ax.add_patch(patches.Rectangle((box[0], box[1]), box[2], box[3], linewidth=1, edgecolor=color, facecolor='none'))

def backprop(b1, b2, loss_fn, lr):
    losses = []
    while loss_fn(b1, b2) > 0.01:
        loss = loss_fn(b1, b2)
        losses.append(loss)
        b1 = b1 - lr * grad(loss_fn)(b1, b2)
        if len(losses) > 2 and losses[-1] == losses[-2]:
            print('Not learning, we quit early.')
            break
    return b1, losses

def run(b1, b2, metric_fn, lr=0.1):
    fig, ax = plt.subplots(1,3, figsize=(15, 5))
    ax[0].set_title('Before Learning')
    plot(b1, 'r', ax[0])
    plot(b2, 'b', ax[0])
    b1, losses = backprop(b1, b2, lambda b1, b2: 1-metric_fn(b1, b2), lr=lr)
    ax[1].set_title('After Learning')
    plot(b1, 'r', ax[1])
    plot(b2, 'b', ax[1])
    ax[2].set_title('Losses over Time')
    ax[2].set_xlabel('Steps')
    ax[2].plot(losses)

## Intersection over Union

In [ ]:
def intersection(b1, b2):
    x1, y1, w1, h1 = b1
    x2, y2, w2, h2 = b2
    xA = max(x1, x2)
    yA = max(y1, y2)
    xB = min(x1 + w1, x2 + w2)
    yB = min(y1 + h1, y2 + h2)
    interArea = max(0, xB - xA) * max(0, yB - yA)
    return interArea

def union(b1, b2):
    x1, y1, w1, h1 = b1
    x2, y2, w2, h2 = b2
    return w1 * h1 + w2 * h2 - intersection(b1, b2)

def iou(b1, b2):
    x1, y1, w1, h1 = b1
    x2, y2, w2, h2 = b2
    return intersection(b1, b2) / union(b1, b2)

b1 = jnp.array([1, 2, 5, 5], dtype=jnp.float32)
b_target = jnp.array([2, 1, 5, 2], dtype=jnp.float32)
run(b1, b_target, iou)

In [ ]:
b2 = jnp.array([4, 6, 1, 1], dtype=jnp.float32)
run(b2, b_target, iou)

## Generalized Intersection over Union

In [ ]:
def enclosement(b1, b2):
    x1, y1, w1, h1 = b1
    x2, y2, w2, h2 = b2
    xA = min(x1, x2)
    yA = min(y1, y2)
    xB = max(x1 + w1, x2 + w2)
    yB = max(y1 + h1, y2 + h2)
    return (xB - xA) * (yB - yA)

def giou(b1, b2):
  R = (enclosement(b1, b2) - union(b1, b2)) / enclosement(b1, b2)
  return iou(b1, b2) - R

run(b1, b_target, giou)

In [ ]:
run(b2, b_target, giou)

## Distance Intersection over Union

In [ ]:
def distance(b1, b2):
    x1, y1, w1, h1 = b1
    x2, y2, w2, h2 = b2
    return jnp.sqrt(jnp.square(x1 - x2) + jnp.square(y1 - y2))

def enclosing_box_diagonal(b1, b2):
    x1, y1, w1, h1 = b1
    x2, y2, w2, h2 = b2
    xA = min(x1, x2)
    yA = min(y1, y2)
    xB = max(x1 + w1, x2 + w2)
    yB = max(y1 + h1, y2 + h2)
    return jnp.sqrt(jnp.square(xB - xA) + jnp.square(yB - yA))

def diou(b1, b2):
    R = jnp.square(distance(b1, b2)) / jnp.square(enclosing_box_diagonal(b1, b2))
    return iou(b1, b2) - R

run(b1, b_target, diou)

In [ ]:
run(b2, b_target, diou)

## Complete Intersection over Union

In [ ]:
def ciou(b1, b2):
    x1, y1, w1, h1 = b1
    x2, y2, w2, h2 = b2
    v = 4 * jnp.square(jnp.arctan(w1 / h1) - jnp.arctan(w2 / h2)) / jnp.square(jnp.pi)
    alpha = v / (1 - iou(b1, b2) + v)
    R = v * alpha
    return diou(b1, b2) - R

run(b1, b_target, ciou)

In [ ]:
run(b2, b_target, ciou)

## Sources
* [Distance-IoU Loss: Faster and Better Learning for Bounding Box Regression](https://arxiv.org/pdf/1911.08287)